# Imports

In [ ]:
import pandas as pd
import numpy as np
import os, glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch

import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, MaxAbsScaler

#plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = (22,16)
plt.rcParams['figure.dpi'] = 100
sns.set(font_scale=3)

# Functions

In [ ]:
def scaler(train, kind='standard', transforms=[]):
    if kind == 'standard':
        scaler = StandardScaler()
    elif kind == 'minmax':
        scaler = MinMaxScaler()
    elif kind == 'maxabs':
        scaler = MaxAbsScaler()
    elif kind == 'quantile':
        scaler = QuantileTransformer()
        
    scaler.fit(train)
    results = []
    for transform in transforms:
        results.append(scaler.transform(transform))
    
    return scaler, results

In [ ]:
def weighted_mean_absolute_percentage_error(true, pred, mean=False):
    true = np.array(true)
    pred = np.array(pred)
    if true.ndim == 1:
        true = true.reshape(1,true.shape[0])
        pred = pred.reshape(1,pred.shape[0])
    wmape = np.abs(pred-true).sum(axis=1) / np.abs(true).sum(axis=1) * 100
    return wmape

In [ ]:
def make_percentile_plots(y_true, y_pred, percentiles, ref_percentile, mses, mapes, wmapes, rows, size, 
                          rows_supylabel, palette='mako'):
    c = sns.color_palette(palette)[0]
    c2 = sns.color_palette(palette)[3]
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    cols = 1
    stks = ['I', 'Q', 'U', 'V']
    kth = 0
    ith_p = []
    
    if len(ref_percentile) != rows:
        models = 1
        for perc in percentiles:
            _, ith = find_nearest(ref_percentile, np.quantile(ref_percentile, perc))
            ith_p.append(ith)
            
        ref_percentile = [ref_percentile]*rows
    
    print(ith_p)
    fig = plt.figure(figsize=size)
    grid = plt.GridSpec(rows, cols)
    
    for i in range(rows * cols):
        fake = fig.add_subplot(grid[i])        
        idx = np.where(np.array(sorted(ref_percentile[i])) == ref_percentile[i][ith_p[i]])
        p = int(np.rint((idx[0][0]+1) / len(ref_percentile[i]) * 100))
        mse = np.round(mses[ith_p[i]], 12)
        mape = np.round(mapes[ith_p[i]], 3)
        wmape = np.round(wmapes[ith_p[i]], 4)
        
        txt = 'Percentil '+str(p)+'\n WMAPE = '+str(wmape)+'\n' 
        fake.set_title(txt, 
                       size=48)
        fake.set_axis_off()

        gs = gridspec.GridSpecFromSubplotSpec(1, 4, subplot_spec=grid[i])

        for col in range(4):
            y_true_col = y_true[ith_p[i]][32*col:32*(col+1)]
            y_pred_col = []
            for ith_model in range(models):
                y_pred_col.append(y_pred[ith_p[0]][32*col:32*(col+1)])
                y_pred_col.append(y_pred[ith_p[1]][32*col:32*(col+1)])
                y_pred_col.append(y_pred[ith_p[2]][32*col:32*(col+1)])
                
            mape = [tf.keras.metrics.mean_absolute_percentage_error(y_true_col, y_pred_col[0]).numpy(), 
                    tf.keras.metrics.mean_absolute_percentage_error(y_true_col, y_pred_col[1]).numpy(),
                    tf.keras.metrics.mean_absolute_percentage_error(y_true_col, y_pred_col[2]).numpy()]
            mse = [tf.keras.metrics.mean_squared_error(y_true_col, y_pred_col[0]).numpy(), 
                    tf.keras.metrics.mean_squared_error(y_true_col, y_pred_col[1]).numpy(),
                    tf.keras.metrics.mean_squared_error(y_true_col, y_pred_col[2]).numpy()]
            wmape = [weighted_mean_absolute_percentage_error(y_true_col, y_pred_col[0]),
                     weighted_mean_absolute_percentage_error(y_true_col, y_pred_col[1]),
                     weighted_mean_absolute_percentage_error(y_true_col, y_pred_col[2])]
            
            text = 'MSE = '+str(np.round(mse[i],16))+'\nWMAPE = '+str(np.round(wmape[i][0], 4))
            ax = fig.add_subplot(gs[col])
            
            sns.lineplot(x=range(32), y=y_true_col, ax=ax, linewidth=6, color=c)
            sns.lineplot(x=range(32), y=y_pred_col[i], ax=ax, linewidth=6, color=c2, linestyle='--')
            ax.text(0.05, 0.95, text, transform=ax.transAxes, fontsize=36, verticalalignment='top', 
                    bbox=props)

            if i == 0:
                ax.set_title(f'Stokes {stks[col]}')
                ax.set_ylabel(f'{stks[col]} / IC',labelpad=0)
            if col == 0:
                ax.annotate(rows_supylabel[i], xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 3, 0), 
                            xycoords=ax.yaxis.label, textcoords='offset points',
                            size='large', ha='right', va='center', rotation='vertical')
    fig.patch.set_facecolor('white')
    fig.tight_layout(w_pad=5, h_pad=1)
    p1 = Patch(facecolor=c, label='Verdad')
    p2 = Patch(facecolor=c2, label='Predicción')

    fig.legend(handles=[p1, p2], loc='lower center', bbox_to_anchor=(0.5,1), 
           ncol=4, bbox_transform=fig.transFigure, fontsize='x-large')

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

In [ ]:
def fully_connected_model(hidden_layers, activation, output, l2):
    model = tf.keras.Sequential()
    for neurons in hidden_layers:
        model.add(tf.keras.layers.Dense(neurons, activation=activation, 
                                        kernel_regularizer=tf.keras.regularizers.l2(l2)))
        
    model.add(tf.keras.layers.Dense(output))
    
    return model

def model_train(x_train, y_train, model, loss, epochs, patience, batch_size, optimizer, 
                verbose, scheduler=None):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    callbacks = [callback]
    if scheduler:
        scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        callbacks.append(scheduler_callback)
            
    model.compile(loss=loss, optimizer=optimizer, metrics=['mae', 'mape', 'mse'])
    history = model.fit(x_train, y_train, epochs=epochs, verbose=verbose, batch_size=batch_size, 
                        validation_split=0.15, callbacks=callbacks)
    
    return model, history

In [ ]:
def load_train_test_set(data_path, test_path, train_rows=None, test_rows=None):
    if train_rows:
        df = pd.read_csv(data_path, nrows=train_rows)
    else:
        df = pd.read_csv(data_path)
        
    if test_rows:
        test_df = pd.read_csv(test_path, nrows=test_rows)
    else:
        test_df = pd.read_csv(test_path)
    return df, test_df

In [ ]:
def load_resolution(df, test_df, all_stk_dim, resolution, xcols, ycols, shift=0):
    mid_point = int(all_stk_dim/2) + shift
    stks = []
    for s in y_cols:
        for i in range(int(mid_point - np.floor(resolution/2)), int(mid_point + np.ceil(resolution/2))):
            stks.append(s+str(i))
    stk_dim = int(len(stks) / 4)
    x = df[xcols]
    y = df[stks]

    xt = test_df[xcols]
    yt = test_df[stks]
    
    return x,y,xt,yt

# Load Data

In [ ]:
data_path = '../data/'
results_path = '../results/'
size_dataset = int(2e6)
test_size_dataset = int(2e5)

new_size_dataset = int(3.5e4)
new_test_size_dataset = int(3.5e3)

In [ ]:
df, test_df = load_train_test_set('../fe6311/cossam_train_data_high.csv', 
                                  '../fe6311/cossam_test_data_high.csv', 
                                  size_dataset, test_size_dataset)

print('Train shape', df.shape)
print('Test shape', test_df.shape)

In [ ]:
new_df, new_test_df = load_train_test_set(data_path+'cossam_train_data_MZS_high.csv', 
                                  data_path+'cossam_test_data_MZS_high.csv', 
                                  new_size_dataset, new_test_size_dataset)

print('Train shape', new_df.shape)
print('Test shape', new_test_df.shape)

# Preprocessing

In [ ]:
cols = ['fmag', 'incl', 'alpha', 'beta', 'gamma', 'y2', 'y3', 'phase']
y_cols = ['stki_' , 'stkq_', 'stku_', 'stkv_']

In [ ]:
x_new_train_normal, y_new_train_normal, x_new_test_normal, y_new_test_normal = load_resolution(new_df,new_test_df, int((new_df.shape[1] - 11)/5), 32, cols, y_cols, shift=0)
x_train_normal, y_train_normal, x_test_normal, y_test_normal = load_resolution(df, test_df, int((df.shape[1] - 11)/5), 32, cols, y_cols)

In [ ]:
x_train_normal = x_train_normal.to_numpy()
x_test_normal = x_test_normal.to_numpy()
y_train_normal = y_train_normal.to_numpy()
y_test_normal = y_test_normal.to_numpy()

x_new_train_normal = x_new_train_normal.to_numpy()
x_new_test_normal = x_new_test_normal.to_numpy()
y_new_train_normal = y_new_train_normal.to_numpy()
y_new_test_normal = y_new_test_normal.to_numpy()

In [ ]:
normal_scalerX, (x_train_s, x_test_s) = scaler(x_train_normal, 'maxabs', 
    [x_train_normal, x_test_normal])
normal_scalerY, (y_train_s, y_test_s) = scaler(y_train_normal, 'standard', 
    [y_train_normal, y_test_normal])

new_normal_scalerX, (x_new_train_s, x_new_test_s) = scaler(x_new_train_normal, 'maxabs', [x_new_train_normal, x_new_test_normal])
new_normal_scalerY, (y_new_train_s, y_new_test_s) = scaler(y_new_train_normal,  'standard', [y_new_train_normal, y_new_test_normal])

In [ ]:
model = tf.keras.models.load_model('new results/1.3M_maxabs_standard_high.h5')

In [ ]:
results = model_stats(model, x_train_s, y_train_normal, x_test_s, 
                      y_test_normal, [normal_scalerY])
_, y_test_pred_base, _, _, _, mse_test_base, _, mape_test_base, wmape_test_base = results
#error_stats_by_stokes(y_new_test_normal, y_test_pred_untrained)
print('Original')
print('---------------------------------')
print('WMAPE:', wmape_test_base.mean(), 'MAPE:', mape_test_base.mean(), ' MSE:', mse_test_base.mean(), '\n')

errors = error_stats_by_stokes(y_test_normal, y_test_pred_base)
for i in range(4):
    print(y_cols[i][:-1], errors['wmape mean'][i+1], '+-', errors['wmape std'][i+1])

In [ ]:
make_percentile_plots(y_test_normal, y_test_pred_base, [0.8, 0.9, 0.99], wmape_test_base,
                      mse_test_base, mape_test_base, wmape_test_base, 3, (48,24), ['']*3)

# Train Last Layer

In [ ]:
tmp_model = tf.keras.models.clone_model(model)
tmp_model.set_weights(model.get_weights())

x = tmp_model.layers[-2].output
predictions = tf.keras.layers.Dense(128, name='output')(x)
transfer_model = tf.keras.models.Model(inputs=tmp_model.input, outputs=predictions)

for i in range(len(transfer_model.layers) - 1):
    transfer_model.layers[i].trainable = False

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=1e-1, momentum=0.95, decay=1/(2*new_size_dataset))
transfer_model, history = model_train(x_new_train_s, y_new_train_s, transfer_model, 'mse', 1000, 25, 1024, opt, 0)

# Finetuning

In [ ]:
finetuning_model = tf.keras.models.clone_model(transfer_model)
finetuning_model.set_weights(transfer_model.get_weights())

s = 6
for i in range(len(finetuning_model.layers) - s):
    finetuning_model.layers[i+s].trainable = True

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.5e-1, momentum=0.95, decay=1/(4*new_size_dataset))
finetuning_model, history = model_train(x_new_train_s, y_new_train_s, finetuning_model, 'mse', 1000, 25, 1024, opt, 1)

In [ ]:
results = model_stats(finetuning_model, x_new_train_s, y_new_train_normal, x_new_test_s, 
                      y_new_test_normal, [new_normal_scalerY])
_, y_test_pred_ft, _, _, _, mse_test_ft, _, mape_test_ft, wmape_test_ft = results

print('Fine Tuning')
print('---------------------------------')
print('WMAPE:', wmape_test_ft.mean(), 'MAPE:', mape_test_ft.mean(), ' MSE:', mse_test_ft.mean(), '\n')

errors = error_stats_by_stokes(y_new_test_normal, y_test_pred_ft)
for i in range(4):
    print(y_cols[i][:-1], errors['wmape mean'][i+1], '+-', errors['wmape std'][i+1])

In [ ]:
wmapes = [np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal))]
mses = [np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal)), np.zeros(len(y_new_test_normal))]
for i in range(4):
    wmapes[i] += weighted_mean_absolute_percentage_error(y_new_test_normal[:,i*32:(i+1)*32], y_test_pred_ft[:,i*32:(i+1)*32])
    mses[i] += tf.keras.metrics.mean_squared_error(y_new_test_normal[:,i*32:(i+1)*32], y_test_pred_ft[:,i*32:(i+1)*32]).numpy()
wmapes[4] = (wmapes[0] + wmapes[1] + wmapes[2] + wmapes[3])/4

resdf = pd.DataFrame({'WMAPE': wmapes[4], 'WMAPE stokes I': wmapes[0], 'WMAPE stokes Q': wmapes[1], 'WMAPE stokes U': wmapes[2], 'WMAPE stokes V': wmapes[3], 'stokes I MSE': mses[0], 'stokes Q MSE': mses[1], 'stokes U MSE': mses[2], 'stokes V MSE': mses[3]})
resdf.to_csv('new results/35k_1.3M_ft_high.csv', index=False)

finetuning_model.save('new results/35k_finetuning_1.3M_high.h5')

In [ ]:
make_percentile_plots(y_new_test_normal, y_test_pred_ft, [0.85, 0.95, 0.99], wmape_test_ft,
                      mse_test_ft, mape_test_ft, wmape_test_ft, 3, (48,24), ['']*3)